# This is the TM-2-WL code applied to the non-isomorphic graph pair $G$ and $H$.

### First, we create all possible 2-tuples.

In [7]:
import numpy as np

t_l = [[(i,j) for i in range(1,13)] for j in range(1,13)]

## This is the fuction that calculates the set of multisets for a $k$-tuple according to definition 8 in the thesis.

In [8]:
def multiset_set_function(input_arr):
    elem_list = [tuple(input_arr)]
    
    for elem in input_arr:
        new_elem = list(input_arr)[:]
        new_elem.append(elem)
        new_elem.sort()
        if len(set(new_elem)) == len(input_arr):
            elem_list.append(tuple(new_elem))
    
    return elem_list

### This is function f by definition 10 that reduces the cardinalities of the vertices within a multiset.

In [10]:
def shrinking_function(input_arr):
    new_list = list(input_arr)[:]
    if len(input_arr) > 2:
        counts = dict() 
        for i in input_arr:
            counts[i] = counts.get(i, 0) + 1
        for key in counts.keys():
            if counts[key] >= 2:
                counts[key] -= 1
        new_list = []
        for key in counts.keys():
            for i in range(counts[key]):
                new_list.append(key)
    return tuple(new_list)


## Multiset Neighbourhood defined as in the paper

In [11]:
def get_neighbourhood_for_multiset(input_multiset):
    if len(input_multiset) == 2:
        all_k_multisets = [(i,j) for i in range(1, 13) for j in range(i, 13)]
    elif len(input_multiset) == 3:
        all_k_multisets = [(i,j,k) for i in range(1,13) for j in range(i,13) for k in range(j, 13)]
    res = []
    for elem in all_k_multisets: 
        if len(set(input_multiset).intersection(set(elem))) == len(input_multiset) - 1:
            if len(set(elem)) <= 2:
                if (len(elem) == 2) or (len(shrinking_function(elem)) == 2):
                    res.append(elem)
    return res

## The following function returns all permutations for a given multiset.

In [12]:
import itertools
def get_permutations(input_multiset):
    all_perms = list(itertools.permutations(input_multiset))    
    return all_perms

## Functions to determine the colour cardinalities after each iteration of the TM-2-WL.

In [13]:
def get_colour_arr(arr, colour_dir):
    res = []
    for elem in arr:
        res.append(colour_dir[elem])
    res.sort()
    return res

In [14]:
def get_colour_counter(dic_colour):
    all_colours = []
    for key_item in dic_colour.keys():
        all_colours.append(dic_colour[key_item])
    res = np.zeros(len(set(all_colours)))
    for elem in all_colours:
        res[elem-1] += 1
    return res

## Encoding of the TM-2-WL algorithm

In [15]:

def TM_k_wl_update(list_tuples, initial_colourings, iterations=10):
    colour_list = [initial_colourings]

    for i in range(iterations):
        colour_counter = get_colour_counter(colour_list[-1])
        print(colour_counter)
        new_colour_list= {}

        injective_func = {}

        for h, list_elements in enumerate(list_tuples):
            for j, elem in enumerate(list_elements):
                new_colour = [colour_list[i][elem]]
                neighbourhoods = []
                multiset_sets = multiset_set_function(elem)
                
                for multiset_elem in multiset_sets: 
                    neighbours_elements = get_neighbourhood_for_multiset(multiset_elem)
                    neighbour_list = []

                    for neighbours in neighbours_elements:
                        new_elem = shrinking_function(neighbours)
                    
                        all_neighbour_perms = get_permutations(new_elem)
                        neighbour_list += all_neighbour_perms

                    neighbour_colour_list = get_colour_arr(neighbour_list, colour_list[i])
                    neighbourhoods.append(tuple(neighbour_colour_list))
               
                new_colour.append(frozenset(tuple(neighbourhoods)))
                new_colour = tuple(new_colour)

                if new_colour not in injective_func.keys():
                    injective_func[new_colour] = len(injective_func)
                

                new_colour_int = injective_func[new_colour]
                new_colour_list[elem] = new_colour_int + 1
        
        colour_list.append(new_colour_list)
    return colour_list[-1], injective_func
                

# Code for graph $G$

## Atomic types for the 2-tuples of $G$.

In [16]:
initial_colourings_G = [
    [2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
    [0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
    [0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 2, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 1, 2, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 1],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 2]]

### This following function maps each atomic type to its according 2-tuple.

In [17]:
Initial_colourings_G = {}
for tupled, colour_list in zip(t_l, initial_colourings_G):
    for t, c in zip(tupled, colour_list):
        Initial_colourings_G[t] = c+1

### Return of the colour cardinalities for graph $G$ for 10 iterations. We can see that after 2 iterations the TM-2-WL has determined a stable colouring for $G$.

In [18]:
colour_list, injective_function = TM_k_wl_update(t_l, initial_colourings=Initial_colourings_G, iterations=10)

[100.  32.  12.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]


# Code for graph $H$

## Atomic types for the 2-tuples of $H$.

In [19]:
initial_colourings_H = [
    [2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
    [0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
    [0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0],
    [1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 2, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 1, 2, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1],
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 2]]

### This following function maps each atomic type to its according 2-tuple.

In [20]:
Initial_colourings_H = {}

for tupled, colour_list in zip(t_l, initial_colourings_H):
    for t, c in zip(tupled, colour_list):
        Initial_colourings_H[t] = c

### Return of the colour cardinalities for graph $H$ for 10 iterations. We can see that after 2 iterations the TM-2-WL has calculated a stable colouring for $H$. Since both stable colourings have been calculated in the same iteration and the colour cardinalities for $G$ and $H$ are the same, we can deduce that the TM-2-WL fails to distinguish $G$ and $H$.

In [21]:
colour_list, injective_function = TM_k_wl_update(t_l, Initial_colourings_H, iterations=10)

[ 32.  12. 100.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
[ 4. 12. 16. 48.  8. 16. 40.]
